# DA 231o Data Engineering at Scale Project



# Expected Goals Model & Player Analysis

### Import Libraries

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import pyspark
pyspark.__version__

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, regexp_replace, when, col

# Load input Dataset with Goals

In [3]:
# Mount the drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Read the csv file for visualization. We do not use this data anywhere in our model

data = pd.read_csv('/content/drive/Shareddrives/DES_Project/events.csv')

In [5]:
print(len(data))
print(data.is_goal.sum())
print(len(data.columns)-1)

941009
24446
21


In [6]:
data.columns

Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break'],
      dtype='object')

In [7]:
# Create spark session

spark = SparkSession \
    .builder \
    .appName('xG Project Solution') \
    .getOrCreate()
  
spark

In [8]:
# Load the events and the info as spark sql dataframes

eventsDF = spark.read.option("header", True).option("inferSchema", True).option("nullable", 'NA').csv('/content/drive/Shareddrives/DES_Project/events.csv')
infoDF = spark.read.option("header", True).option("inferSchema", True).option("nullable", 'NA').csv('/content/drive/Shareddrives/DES_Project/ginf.csv')

In [9]:
# Print Schema

eventsDF.printSchema()

infoDF.printSchema()

root
 |-- id_odsp: string (nullable = true)
 |-- id_event: string (nullable = true)
 |-- sort_order: integer (nullable = true)
 |-- time: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- event_type: integer (nullable = true)
 |-- event_type2: string (nullable = true)
 |-- side: integer (nullable = true)
 |-- event_team: string (nullable = true)
 |-- opponent: string (nullable = true)
 |-- player: string (nullable = true)
 |-- player2: string (nullable = true)
 |-- player_in: string (nullable = true)
 |-- player_out: string (nullable = true)
 |-- shot_place: string (nullable = true)
 |-- shot_outcome: string (nullable = true)
 |-- is_goal: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- bodypart: string (nullable = true)
 |-- assist_method: integer (nullable = true)
 |-- situation: string (nullable = true)
 |-- fast_break: integer (nullable = true)

root
 |-- id_odsp: string (nullable = true)
 |-- link_odsp: string (nullable = true)
 |-- adv_stat

In [10]:
# Verify if the data loaded is correct and in the proper format

eventsDF.show(10)
type(eventsDF)

+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-------------------+----------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+
|  id_odsp|  id_event|sort_order|time|                text|event_type|event_type2|side|       event_team|         opponent|             player|         player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|
+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-------------------+----------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+
|UFot0hit/| UFot0hit1|         1|   2|Attempt missed. M...|         1|         12|   2|       Hamburg SV|Borussia Dortmund|      mladen petric|     gokhan tore|       NA|        NA|         6|       

pyspark.sql.dataframe.DataFrame

In [11]:
# Join id_odsp, country, date columns from infoDF on id_odsp

eventsDF = eventsDF.join(infoDF[['id_odsp', 'country', 'date']], on='id_odsp', how='left')
eventsDF.printSchema()
eventsDF.show(10)
eventsDF.count()

root
 |-- id_odsp: string (nullable = true)
 |-- id_event: string (nullable = true)
 |-- sort_order: integer (nullable = true)
 |-- time: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- event_type: integer (nullable = true)
 |-- event_type2: string (nullable = true)
 |-- side: integer (nullable = true)
 |-- event_team: string (nullable = true)
 |-- opponent: string (nullable = true)
 |-- player: string (nullable = true)
 |-- player2: string (nullable = true)
 |-- player_in: string (nullable = true)
 |-- player_out: string (nullable = true)
 |-- shot_place: string (nullable = true)
 |-- shot_outcome: string (nullable = true)
 |-- is_goal: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- bodypart: string (nullable = true)
 |-- assist_method: integer (nullable = true)
 |-- situation: string (nullable = true)
 |-- fast_break: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- date: timestamp (nullable = true)

+---------+---------

941009

In [12]:
# Extract year from the timestamp and append the column

extract_year = lambda x: x.strftime("%Y")
extract_year_udf = udf(extract_year)
eventsDF = eventsDF.withColumn("year", extract_year_udf("date"))
eventsDF.show(10)

+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-------------------+----------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+
|  id_odsp|  id_event|sort_order|time|                text|event_type|event_type2|side|       event_team|         opponent|             player|         player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|country|               date|year|
+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-------------------+----------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+
|UFot0hit/| UFot0hit1|         1|   2|Attempt missed. M...|         1|         12|   2|       Hambur

In [13]:
eventsDF.filter(eventsDF.event_type==1).filter(eventsDF.player != '').count()

229134

In [14]:
# Create shotsDF dataframe by filtering event type == 1 and non-empty player

shotsDF = eventsDF.filter(eventsDF.event_type==1).filter(eventsDF.player != '').filter(eventsDF.player2 != '').filter(eventsDF.country != '')
shotsDF = shotsDF.withColumn("player", udf(lambda x: x.title())("player"))
shotsDF = shotsDF.withColumn("player2", udf(lambda x: x.title())("player2"))
shotsDF = shotsDF.withColumn("country", udf(lambda x: x.title())("country"))
shotsDF.show(10)

+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-----------------+------------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+
|  id_odsp|  id_event|sort_order|time|                text|event_type|event_type2|side|       event_team|         opponent|           player|           player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|country|               date|year|
+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-----------------+------------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+
|UFot0hit/| UFot0hit1|         1|   2|Attempt missed. M...|         1|         12|   2|       Hambur

In [15]:
# Filter the columns which have all the relevant information about the shot and predict if it is a goal or not
# is_goal is the true label. We will not use this in the training

required_cols = ['shot_place', 'shot_outcome', 'location', 'bodypart', 'assist_method', 'situation']

# Create dataDF dataframe with all the input columns
dataDF = shotsDF

# Create string indexer list
indexer = []

# Convert each column into integer type using string indexer without losing any features
for column in required_cols:
  temp = StringIndexer(inputCol=column, outputCol=column+"_index")
  indexer.append(temp)

# Pipeline
pipeline = Pipeline(stages=indexer)
dataDF = pipeline.fit(dataDF).transform(dataDF)

dataDF.show(10)

+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-----------------+------------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+----------------+------------------+--------------+--------------+-------------------+---------------+
|  id_odsp|  id_event|sort_order|time|                text|event_type|event_type2|side|       event_team|         opponent|           player|           player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|country|               date|year|shot_place_index|shot_outcome_index|location_index|bodypart_index|assist_method_index|situation_index|
+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-----------------+------------------+---------+----------+----------+----

# Expected Goals with PySpark Machine Learning

In [16]:
required_features = list(dataDF.columns[-6:]) # remove 'is_goal' because it is the label

assembler = VectorAssembler(inputCols=required_features, outputCol='features')
transformed_data = assembler.transform(dataDF)

transformed_data.show(10)

+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-----------------+------------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+----------------+------------------+--------------+--------------+-------------------+---------------+--------------------+
|  id_odsp|  id_event|sort_order|time|                text|event_type|event_type2|side|       event_team|         opponent|           player|           player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|country|               date|year|shot_place_index|shot_outcome_index|location_index|bodypart_index|assist_method_index|situation_index|            features|
+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+-----------------+-------------

In [17]:
# Split the data into training and testing data

(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

print("Total Data", transformed_data.count())
print("Total Training Data", training_data.count())
print("Total Test Data", test_data.count())

print("\nTraining Dataset")
training_data.show(5)

Total Data 229131
Total Training Data 183268
Total Test Data 45863
Training Dataset
+---------+-----------+----------+----+--------------------+----------+-----------+----+----------+--------+------------------+---------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+----------------+------------------+--------------+--------------+-------------------+---------------+--------------------+
|  id_odsp|   id_event|sort_order|time|                text|event_type|event_type2|side|event_team|opponent|            player|        player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|country|               date|year|shot_place_index|shot_outcome_index|location_index|bodypart_index|assist_method_index|situation_index|            features|
+---------+-----------+----------+----+--------------------+----------+-----------+----+----------+-----

# Gradient-Boosted Trees (GBTs) Classifier

In [18]:
gbt = GBTClassifier(labelCol='is_goal', 
                            featuresCol='features',
                            maxDepth=5)

In [20]:
# Another classifier
# from pyspark.ml.classification import DecisionTreeClassifier
# gbt = DecisionTreeClassifier(labelCol='is_goal', 
#                             featuresCol='features',
#                             maxDepth=5)

In [21]:
# Fit the model on training data
model = gbt.fit(training_data)

# Predictions with Test Data

In [22]:
# Predictions with test data
predictions = model.transform(test_data)
predictions.show(5)

+---------+-----------+----------+----+--------------------+----------+-----------+----+----------+--------+------------------+----------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+----------------+------------------+--------------+--------------+-------------------+---------------+--------------------+--------------------+--------------------+----------+
|  id_odsp|   id_event|sort_order|time|                text|event_type|event_type2|side|event_team|opponent|            player|         player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|country|               date|year|shot_place_index|shot_outcome_index|location_index|bodypart_index|assist_method_index|situation_index|            features|       rawPrediction|         probability|prediction|
+---------+-----------+----------+----+--------------------+----------+---------

# Evaluate our model

In [23]:
# Evaluate our model
evaluator = MulticlassClassificationEvaluator(
    labelCol='is_goal', 
    predictionCol='prediction', 
    metricName='accuracy')

# Calculate Test Accuracy

In [24]:
accuracy = evaluator.evaluate(predictions)
print('Test Accuracy = ', accuracy)

Test Accuracy =  0.9373787148681944


In [25]:
from pyspark.ml.evaluation import RegressionEvaluator

regEvaluator = RegressionEvaluator(
    labelCol='is_goal', 
    predictionCol='prediction', 
    metricName='rmse')

rmse = regEvaluator.evaluate(predictions)
print('RMSE = ', rmse)

RMSE =  0.25024245269699064


# Player Analysis

In [26]:
# Predict the expected goal on all the data (training + test)

pred = model.transform(transformed_data)
pred.show()
pred.count()

+---------+----------+----------+----+--------------------+----------+-----------+----+-----------------+-----------------+------------------+------------------+---------+----------+----------+------------+-------+--------+--------+-------------+---------+----------+-------+-------------------+----+----------------+------------------+--------------+--------------+-------------------+---------------+--------------------+--------------------+--------------------+----------+
|  id_odsp|  id_event|sort_order|time|                text|event_type|event_type2|side|       event_team|         opponent|            player|           player2|player_in|player_out|shot_place|shot_outcome|is_goal|location|bodypart|assist_method|situation|fast_break|country|               date|year|shot_place_index|shot_outcome_index|location_index|bodypart_index|assist_method_index|situation_index|            features|       rawPrediction|         probability|prediction|
+---------+----------+----------+----+--------

229131

In [27]:
# Create a new dataframe with selected columns for player analysis

players = pred.select('player', 'is_goal', 'prediction')
players.show(5)
players.count()

+-----------------+-------+----------+
|           player|is_goal|prediction|
+-----------------+-------+----------+
|    Mladen Petric|      0|       0.0|
|    Shinji Kagawa|      0|       0.0|
|Kevin Grosskreutz|      1|       0.0|
|     Mats Hummels|      0|       0.0|
|     Tomas Rincon|      0|       0.0|
+-----------------+-------+----------+
only showing top 5 rows



229131

In [28]:
# Count the number of actual goals and expected goals per player

player_group = players.groupBy('player').sum()
player_group = player_group.withColumnRenamed('sum(is_goal)', 'true_goals').withColumnRenamed('sum(prediction)', 'expected_goals')

# Calculate the difference between the expected goals and the true goals
player_group = player_group.withColumn('difference', player_group.expected_goals - player_group.true_goals)
player_group.show(5)

+----------------+----------+--------------+----------+
|          player|true_goals|expected_goals|difference|
+----------------+----------+--------------+----------+
|    Isimat Mirin|         0|           0.0|       0.0|
| Claudio Pizarro|        47|          48.0|       1.0|
|   Julien Escude|         1|           2.0|       1.0|
|Antonio Da Silva|         0|           0.0|       0.0|
|     Inigo Lopez|         5|           5.0|       0.0|
+----------------+----------+--------------+----------+
only showing top 5 rows



In [43]:
# Sort by the least difference (Expected - True)
# Absolute difference indicates the number of goals made on top of the expected goals

# Best Finishers with most goals on top of expected
result = player_group.sort('difference')
result.show(6)

+------------------+----------+--------------+----------+
|            player|true_goals|expected_goals|difference|
+------------------+----------+--------------+----------+
| Cristiano Ronaldo|       198|         168.0|     -30.0|
|   Gonzalo Higuain|       118|          95.0|     -23.0|
|        Marco Reus|        65|          44.0|     -21.0|
|       Gareth Bale|        50|          31.0|     -19.0|
|Robert Lewandowski|       124|         105.0|     -19.0|
|      Lionel Messi|       205|         186.0|     -19.0|
+------------------+----------+--------------+----------+
only showing top 6 rows



In [44]:
# Messi is my favourite player. So showing the result below

result.filter(result.player == 'Lionel Messi').show()

+------------+----------+--------------+----------+
|      player|true_goals|expected_goals|difference|
+------------+----------+--------------+----------+
|Lionel Messi|       205|         186.0|     -19.0|
+------------+----------+--------------+----------+



In [45]:
# Sort by ratio between the true goals and the expected goals in descending order

# Best Finishers with highest (true goals / expected goals) ratio
result2 = player_group.withColumn('ratio', player_group['true_goals'] / player_group['expected_goals'])
result2.orderBy(col('ratio').desc()).show(5)

+---------------+----------+--------------+----------+-----+
|         player|true_goals|expected_goals|difference|ratio|
+---------------+----------+--------------+----------+-----+
|   Gokhan Inler|         9|           1.0|      -8.0|  9.0|
| Cristian Tello|        13|           2.0|     -11.0|  6.5|
|   Jordon Mutch|         6|           1.0|      -5.0|  6.0|
|Geoffrey Dernis|         6|           1.0|      -5.0|  6.0|
| Adrian Embarba|         5|           1.0|      -4.0|  5.0|
+---------------+----------+--------------+----------+-----+
only showing top 5 rows



In [31]:
pred.filter(pred.prediction == 1).count()

21484